In [1]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
from config import * #some main key parameters needed.

### 2. Get the Global Steel Plant Tracker database:

In [2]:
db_file_gb_plants = pd.read_excel("data\Global-Steel-Plant-Tracker-April-2024-Standard-Copy-V1.xlsx", 
                                  sheet_name='Steel Plants')
db_file_gb_production = pd.read_excel("data\Global-Steel-Plant-Tracker-April-2024-Standard-Copy-V1.xlsx", 
                                  sheet_name='Yearly Production', header=1)
# Merge the DataFrames based on 'Plant ID'
db_file_gb_merged_init = pd.merge(db_file_gb_production, db_file_gb_plants, on='Plant ID', how='inner')
db_file_gb_merged_init

,Plant ID,Plant name (English)_x,Crude steel production 2019 (ttpa),BOF steel production 2019 (ttpa),EAF steel production 2019 (ttpa),OHF steel production 2019 (ttpa),Iron production 2019 (ttpa),BF production 2019 (ttpa),DRI production 2019 (ttpa),Crude steel production 2020 (ttpa),...,Workforce size,ISO 14001,ISO 50001,ResponsibleSteel Certification,Main production process,Main production equipment,Detailed production equipment,Power source,Iron ore source,Met coal source
0,P100000120001,GHC Emirates Steel Industries Abu Dhabi plant,unknown,NaN,unknown,NaN,unknown,NaN,unknown,unknown,...,3000,yes,NaN,NaN,integrated (DRI),"DRI, EAF",2 DRI plant ( 1 announced) ; 3 EAF plants,NaN,NaN,NaN
1,P100000120004,Kurum International Elbasan steel plant,unknown,NaN,unknown,NaN,unknown,unknown,NaN,unknown,...,1000,yes,NaN,NaN,electric,EAF,1 EAF (began in 2010),"Hydraulic, integrated plants",NaN,NaN
2,P100000120005,Aceria Angola Bengo steel plant,unknown,NaN,unknown,NaN,NaN,NaN,NaN,unknown,...,600,unknown,unknown,NaN,steelmaking (other),other,1 IF,NaN,NaN,NaN
3,P100000120006,ArcelorMittal Acindar Villa Constitución steel...,1100,NaN,1100,NaN,unknown,NaN,unknown,800,...,1589,2020,NaN,NaN,integrated (DRI),"DRI, EAF",1 DRI plant: 5.5-meter MIDREX CDRI Shaft Furna...,Signed agreement with Petroquimica Comodoro Ri...,NaN,NaN
4,P100000120007,TenarisSiderca Campana steel plant,878,NaN,878,NaN,unknown,NaN,unknown,694,...,unknown,NaN,NaN,NaN,electric,EAF,EAF #4: Tenova (1995),onsite thermoelectric power plant (since 1963)...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,P100000121240,Donalam Targoviste plant,unknown,NaN,unknown,NaN,NaN,NaN,NaN,unknown,...,730,yes,NaN,NaN,electric,EAF,1 EAF (75-tonne),NaN,NaN,NaN
1647,P100000121240,Donalam Targoviste plant,unknown,NaN,unknown,NaN,NaN,NaN,NaN,unknown,...,270,NaN,NaN,NaN,electric,EAF,1 EAF,75MW PV park,NaN,NaN
1648,P100000121242,Silcotub Calarasi steel mill,unknown,NaN,unknown,NaN,NaN,NaN,NaN,unknown,...,908,unknown,unknown,NaN,electric,EAF,1 EAF,NaN,NaN,NaN
1649,P100000121248,Al Qaryan steel plant,unknown,NaN,NaN,NaN,NaN,NaN,NaN,unknown,...,unknown,unknown,unknown,NaN,steelmaking (other),other,IF,NaN,NaN,NaN


In [3]:
prod_capacity_columns_steel = ['Nominal BOF steel capacity (ttpa)',	'Nominal EAF steel capacity (ttpa)', 'Nominal OHF steel capacity (ttpa)']
prod_capacity_columns_iron = ['Nominal BF capacity (ttpa)',	'Nominal DRI capacity (ttpa)']

# Function to get the most recent valid production value for each plant
def get_latest_production(row, columns_of_interest):
    # Loop through the columns of interest from latest to earliest year
    for col in reversed(columns_of_interest):
        if row[col] not in ['unknown', np.nan]:  # Check if data is not 'unknown' or NaN
            return row[col]
    return np.nan  # If all years have missing or 'unknown' data, return NaN

# List of steel production types
cols_to_assess = [#'Crude steel production', 
                  'BOF steel production', 'EAF steel production', 'EAF steel production (DRI)',
                  'OHF steel production', 'EAF steel production (DRI, NG)', 'EAF steel production (DRI, coal)',
                  #'Iron production', 
                'BF production', 'DRI production']

# Iterate over each production type and apply the function
for col in cols_to_assess:
    # Filter the columns that relate to the specific steel type and production years
    production_columns = db_file_gb_merged_init.filter(like=col).filter(like='ttpa').columns

    # Apply the function to get the latest production data
    db_file_gb_merged_init[f"{col}"] = db_file_gb_merged_init.apply(
        get_latest_production, columns_of_interest=production_columns, axis=1
    )

db_file_gb_merged_init[cols_to_assess] = db_file_gb_merged_init[cols_to_assess].apply(pd.to_numeric, errors='coerce').divide(1e3)
# Split the 'Coordinates' column into 'Latitude' and 'Longitude'
db_file_gb_merged_init[['Latitude', 'Longitude']] = db_file_gb_merged_init['Coordinates'].str.split(',', expand=True)

# Convert the new columns to numeric, handling any non-numeric values like 'unknown'
db_file_gb_merged_init['Latitude'] = pd.to_numeric(db_file_gb_merged_init['Latitude'], errors='coerce')
db_file_gb_merged_init['Longitude'] = pd.to_numeric(db_file_gb_merged_init['Longitude'], errors='coerce')

info_cols = ['Plant ID', 'Plant name (English)_x', 'Country/Area', 'Power source', 'Start date', 
             'Capacity operating status','Retired Date', 'Latitude', 'Longitude', 'Category steel product','Steel products',
             'Main production process', 'Main production equipment'] + prod_capacity_columns_steel + prod_capacity_columns_iron + cols_to_assess

db_file_gb_merged_init = db_file_gb_merged_init[info_cols]
db_file_gb_merged_init

,Plant ID,Plant name (English)_x,Country/Area,Power source,Start date,Capacity operating status,Retired Date,Latitude,Longitude,Category steel product,...,Nominal BF capacity (ttpa),Nominal DRI capacity (ttpa),BOF steel production,EAF steel production,EAF steel production (DRI),OHF steel production,"EAF steel production (DRI, NG)","EAF steel production (DRI, coal)",BF production,DRI production
0,P100000120001,GHC Emirates Steel Industries Abu Dhabi plant,United Arab Emirates,NaN,2003,operating,NaN,24.322033,54.467987,semi-finished; finished rolled,...,NaN,4000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P100000120004,Kurum International Elbasan steel plant,Albania,"Hydraulic, integrated plants",1998,operating,NaN,41.094611,20.026117,semi-finished; finished rolled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P100000120005,Aceria Angola Bengo steel plant,Angola,NaN,2015,operating,NaN,-8.582400,13.405271,semi-finished; finished rolled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P100000120006,ArcelorMittal Acindar Villa Constitución steel...,Argentina,Signed agreement with Petroquimica Comodoro Ri...,1946,operating,NaN,-33.249936,-60.300415,finished rolled,...,NaN,600,NaN,1.174,NaN,NaN,NaN,NaN,NaN,0.551
4,P100000120007,TenarisSiderca Campana steel plant,Argentina,onsite thermoelectric power plant (since 1963)...,NaN,operating pre-retirement,unknown,-34.152554,-58.977388,finished rolled,...,NaN,NaN,NaN,0.922,NaN,NaN,NaN,NaN,NaN,0.882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,P100000121240,Donalam Targoviste plant,Romania,NaN,1973,operating,NaN,44.904569,25.458124,finished rolled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1647,P100000121240,Donalam Targoviste plant,Romania,75MW PV park,2024,construction,NaN,44.904569,25.458124,finished rolled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1648,P100000121242,Silcotub Calarasi steel mill,Romania,NaN,unknown,operating,NaN,44.217275,27.299680,semi-finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1649,P100000121248,Al Qaryan steel plant,Saudi Arabia,NaN,unknown,operating,NaN,26.280741,49.956755,semi-finished; finished rolled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### If we don't have information on the latest production of crude steel or iron, we base it on the capacity to do so:

In [4]:
# Get the plants only in operation
db_file_gb_merged = db_file_gb_merged_init.copy()

In [5]:
# Define columns to sum
columns_to_sum = prod_capacity_columns_steel + prod_capacity_columns_iron 
columns_to_mean = cols_to_assess

db_file_gb_merged[columns_to_sum] = db_file_gb_merged[columns_to_sum].apply(pd.to_numeric, errors='coerce')

# Group by and aggregate
db_file_gb_merged = db_file_gb_merged.groupby(['Plant ID'], as_index=False).agg(
    {**{col: 'sum' for col in columns_to_sum}, 
    **{col: 'mean' for col in columns_to_mean}, 
     **{col: 'first' for col in db_file_gb_merged.columns if col not in columns_to_sum and col != 'Plant ID'}}
)
db_file_gb_merged

,Plant ID,Nominal BOF steel capacity (ttpa),Nominal EAF steel capacity (ttpa),Nominal OHF steel capacity (ttpa),Nominal BF capacity (ttpa),Nominal DRI capacity (ttpa),BOF steel production,EAF steel production,EAF steel production (DRI),OHF steel production,...,Power source,Start date,Capacity operating status,Retired Date,Latitude,Longitude,Category steel product,Steel products,Main production process,Main production equipment
0,P100000120001,0.0,3600.0,0.0,0.0,4000.0,NaN,NaN,NaN,NaN,...,None,2003,operating,None,24.322033,54.467987,semi-finished; finished rolled,"rebar, wire rod, heavy sections, sheet piles, ...",integrated (DRI),"DRI, EAF"
1,P100000120004,0.0,700.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,"Hydraulic, integrated plants",1998,operating,None,41.094611,20.026117,semi-finished; finished rolled,billets; rebar; wire rods; spooler,electric,EAF
2,P100000120005,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,None,2015,operating,None,-8.582400,13.405271,semi-finished; finished rolled,rebar; wire rod; billet; wire mesh,steelmaking (other),other
3,P100000120006,0.0,2000.0,0.0,0.0,600.0,NaN,1.174,NaN,NaN,...,Signed agreement with Petroquimica Comodoro Ri...,1946,operating,None,-33.249936,-60.300415,finished rolled,"wire rod, bar, steel mesh, wires, turnbuckles,...",integrated (DRI),"DRI, EAF"
4,P100000120007,0.0,2250.0,0.0,0.0,960.0,NaN,0.922,NaN,NaN,...,onsite thermoelectric power plant (since 1963)...,1962,operating pre-retirement,unknown,-34.152554,-58.977388,finished rolled,seamless steel pipes,electric,EAF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,P100000121238,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,None,2021-03,operating,None,22.511230,91.719806,semi-finished; finished rolled,"grade rods, angles, square bars, flat bars, ch...",unknown,unknown
1000,P100000121239,0.0,0.0,0.0,73.0,183.0,NaN,NaN,NaN,NaN,...,Waste heat recover captive power plants (12 MW...,1990,operating,None,23.312265,70.201953,crude; semi-finished; finished rolled,sponge iron; pig iron; billets; bars,integrated (BF and DRI),"BF, DRI, other"
1001,P100000121240,0.0,800.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,75MW PV park,1973,operating,None,44.904569,25.458124,finished rolled,rebar; wire rod,electric,EAF
1002,P100000121242,0.0,535.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,None,unknown,operating,None,44.217275,27.299680,semi-finished,blooms; billet,electric,EAF


In [6]:
prod_capacity_columns_steel

['Nominal BOF steel capacity (ttpa)',
 'Nominal EAF steel capacity (ttpa)',
 'Nominal OHF steel capacity (ttpa)']

In [7]:
# Define your steel production columns and corresponding capacity columns
cols_to_assess_steel = ['BOF steel production', 'EAF steel production', 'OHF steel production']
prod_capacity_columns_steel = ['Nominal BOF steel capacity (ttpa)', 'Nominal EAF steel capacity (ttpa)', 'Nominal OHF steel capacity (ttpa)']  

# Define capacity factors for each type
cf_dict = {
    'BOF steel production': 0.80,
    'EAF steel production': 0.70,
    'OHF steel production': 0.50
}

# Ensure numeric types for capacity columns
db_file_gb_merged[prod_capacity_columns_steel] = db_file_gb_merged[prod_capacity_columns_steel].apply(pd.to_numeric, errors='coerce')

# Find rows where all production values are zero or NaN
all_zero_or_nan = (db_file_gb_merged[cols_to_assess_steel].fillna(0) == 0).all(axis=1)

# Loop over each production column and apply respective CF * capacity
for prod_col, cap_col in zip(cols_to_assess_steel, prod_capacity_columns_steel):
    cf = cf_dict[prod_col]
    db_file_gb_merged.loc[all_zero_or_nan, prod_col] = (
        db_file_gb_merged.loc[all_zero_or_nan, cap_col] / 1e3 * cf
    )

In [8]:
# Define columns and capacity factors
cols_to_assess_iron = ['BF production', 'DRI production']
prod_capacity_columns_iron = ['Nominal BF capacity (ttpa)', 'Nominal DRI capacity (ttpa)']  # adjust names if needed

cf_dict_iron = {
    'BF production': 0.80,   # Blast Furnace
    'DRI production': 0.70   # Direct Reduced Iron
}

# Ensure numeric types
db_file_gb_merged[cols_to_assess_iron] = db_file_gb_merged[cols_to_assess_iron].apply(pd.to_numeric, errors='coerce')
db_file_gb_merged[prod_capacity_columns_iron] = db_file_gb_merged[prod_capacity_columns_iron].apply(pd.to_numeric, errors='coerce')

# Find rows where all production values are zero or NaN
all_zero_or_nan = (db_file_gb_merged[cols_to_assess_iron].fillna(0) == 0).all(axis=1)

# Fill missing values based on individual capacity * CF
for prod_col, cap_col in zip(cols_to_assess_iron, prod_capacity_columns_iron):
    cf = cf_dict_iron[prod_col]
    db_file_gb_merged.loc[all_zero_or_nan, prod_col] = (
        db_file_gb_merged.loc[all_zero_or_nan, cap_col] / 1e3 * cf
    )


In [9]:
plants_mod = db_file_gb_merged.copy()

In [10]:
duplicates = plants_mod[plants_mod.duplicated(subset=['Plant ID'], keep=False)]
duplicates

,Plant ID,Nominal BOF steel capacity (ttpa),Nominal EAF steel capacity (ttpa),Nominal OHF steel capacity (ttpa),Nominal BF capacity (ttpa),Nominal DRI capacity (ttpa),BOF steel production,EAF steel production,EAF steel production (DRI),OHF steel production,...,Power source,Start date,Capacity operating status,Retired Date,Latitude,Longitude,Category steel product,Steel products,Main production process,Main production equipment


In [11]:
mask = (plants_mod['Main production process'] == 'integrated (DRI)') & \
       (plants_mod['Main production equipment'] == 'DRI, EAF')
mask

0        True
1       False
2       False
3        True
4       False
        ...  
999     False
1000    False
1001    False
1002    False
1003    False
Length: 1004, dtype: bool

In [12]:
# Create new column with the values
plants_mod.loc[mask, 'EAF steel production (DRI, NG)'] = plants_mod.loc[mask, 'EAF steel production']

# Set the original column to zero for these rows
plants_mod.loc[mask, 'EAF steel production'] = 0
plants_mod[(plants_mod['Main production process'] == 'integrated (DRI)') & (plants_mod['Main production equipment'] == 'DRI, EAF')]['EAF steel production (DRI, NG)'].sum()

80.12970000000001

## Next, use fuzzy logic to get the most likely power source:

In [13]:
def classify_power_type(power_description):
    """Classifies power sources into PV, Wind, Nuclear, Coal, Natural Gas, or Grid using fuzzy matching."""
    categories = {
            "pv": ["photovoltaic", "solar", "rooftop pv", "solar farm", "solar power", "solar plant", "solar panels", "photovoltaics"],
            "wind": ["wind farm", "windseeg", "wind", "wind turbine", "onshore wind", "wind energy"],
            "nuclear": ["nuclear", "nuclear power", "nuclear plant", "atomic energy"],
            "coal": ["coal", "coal-fired", "coal power", "coal plant", "lignite", "hard coal"],
            "natural gas": ["natural gas", "gas-fired", "gas plant", "ng", "cng", "lng", "combined cycle gas turbine", "ccgt"],
            "grid": ["grid", "power supply", "state grid", "electric grid", "national grid", "grid electricity", "utility power", "captive power plant", "power plant"],
            "renewable": ["renewable", "renewable energy", "green energy", "clean energy", "sustainable energy"]
            }
    
    power_description = str(power_description).lower()
    
    best_match = None
    highest_score = 0
    
    for category, keywords in categories.items():
        #print(process.extractOne(power_description, keywords))
        match, score, __ = process.extractOne(power_description, keywords)
        if score > highest_score:
            highest_score = score
            best_match = category
    
    return best_match if highest_score > 85 else "grid"  # Default to Grid if no strong match

plants_mod["power_classification"] = plants_mod["Power source"].apply(classify_power_type)

# Now, we consider that normal DRI is made using Natural gas, however, in India, they typical use coal and we can consider this
see: https://publications.jrc.ec.europa.eu/repository/bitstream/JRC129297/JRC129297_01.pdf 

In [14]:
mask = (plants_mod['Main production process'] == 'integrated (DRI)') & \
       (plants_mod['Main production equipment'] == 'DRI, EAF') & \
       (plants_mod['Country/Area'] == 'India')
mask.value_counts()

# Create new column with the values
plants_mod.loc[mask, 'EAF steel production (DRI, coal)'] = plants_mod.loc[mask, 'EAF steel production (DRI, NG)']

# Set the original column to zero for these rows
plants_mod.loc[mask, 'EAF steel production (DRI, NG)'] = 0
plants_mod[(plants_mod['Main production process'] == 'integrated (DRI)') & (plants_mod['Main production equipment'] == 'DRI, EAF')]['EAF steel production (DRI, coal)'].sum()

1.274

In [15]:
def categorize_production(row):
    # Check if there is steel production (non-null and > 0)
    steel_production = any(row[col] > 0 for col in ['BOF steel production', 'EAF steel production', 'OHF steel production', 'EAF steel production (DRI, coal)', 'EAF steel production (DRI, NG)'] 
                           if pd.notnull(row[col]))
    
    # Check if there is iron production (non-null and > 0)
    iron_production = any(row[col] > 0 for col in ['BF production', 'DRI production'] 
                          if pd.notnull(row[col]))
    
    if steel_production and iron_production:
        return 'both'
    elif steel_production:
        return 'steel'
    elif iron_production:
        return 'iron'
    else:
        return 'none'

# Create a new column 'Steel_Iron_Production' based on the presence of production values
plants_mod['steel_or_iron_production'] = plants_mod.apply(categorize_production, axis=1)
plants_mod = plants_mod[plants_mod['steel_or_iron_production'] != 'none']
plants_mod[cols_to_assess].sum()

BOF steel production                1769.7338
EAF steel production                 519.6595
EAF steel production (DRI)             0.0000
OHF steel production                   2.6150
EAF steel production (DRI, NG)        78.8557
EAF steel production (DRI, coal)       1.2740
BF production                       1577.0534
DRI production                       155.7738
dtype: float64

In [16]:
print(plants_mod['EAF steel production (DRI)'].sum()/plants_mod[cols_to_assess].sum().sum())
plants_mod = plants_mod.drop(columns=['EAF steel production (DRI)'])

0.0


In [17]:
def classify_steel(steel_product):
    """
    Classifies steel products into:
        - steel, low-alloyed, hot rolled
        - steel, unalloyed
        - steel, low-alloyed
    using fuzzy matching to find the best match.
    """

    # Steel categories with relevant keywords
    steel_categories = {
        "unalloyed": [
            "carbon", "unalloyed", "un-alloyed", "mild", "construction", "black",
            "boiler plate", "pipe", "sheet", "strip", 'flat', 'billet','tube'
        ],
        "low-alloyed": [
            "low alloyed", "low-alloyed", "low-alloy", "low alloy", "high strength low alloy", 
            "HSLA", "quenched and tempered", "tempered", 'stainless',"high strength", "high-strength"
            "chrome-molybdenum", "Cr-Mo"
        ],
        #"low-alloyed, hot rolled": ["hot re-rolled", "hot rolled", "merchant bars", "beams", "sections", "angles", "U channels", "corrugated bars", "round bars", "wire rods", "square bars", "helical rounds"]
    }

    steel_product_lower = str(steel_product).lower()  # Ensure string format
    
    if steel_product_lower == 'unknown':
        return "unalloyed"  # Assume unknown to be unalloyed as most steel is unalloyed.

    # Apply fuzzy matching to get the best match
    match_value, score, best_match_key = process.extractOne(
        steel_product_lower, 
        {key: " ".join(val) for key, val in steel_categories.items()},
        scorer=fuzz.partial_ratio
    )

    # If the best match is "Low-Alloyed" and "hot rolled" is in the product name, override to "Low-Alloyed, Hot Rolled"
    #if best_match_key == "low-alloyed" and "hot rolled" in steel_product_lower:
    #    return "low-alloyed, hot rolled"

    return best_match_key  # Return the best category match

# Apply classification to DataFrame
plants_mod["steel_classification"] = plants_mod["Steel products"].apply(classify_steel)
plants_mod["steel_classification"].value_counts()

steel_classification
unalloyed      824
low-alloyed    170
Name: count, dtype: int64

### 2. Add low-carbon projects:
data from: https://www.industrytransition.org/green-steel-tracker/
LeadIT. (2024). Green Steel Tracker. Leadership Group for Industry Transition. https://www.industrytransition.org/green-steel-tracker/
Lastly updated: December, 2024

In [18]:
db_file_lc_projects = pd.read_excel("data\low_carbon_steel_projects.xlsx", 
                                  sheet_name='1.2 Consolidate (1+1.1)', skiprows=0)

In [19]:
def classify_decarb_type(decarb_description):
    """
    Classifies the given decarbonization description into categories based on fuzzy matching.
    
    The function compares the provided `decarb_description` with predefined keywords for 
    different decarbonization technologies such as CCS (Carbon Capture and Storage) and 
    DRI (Direct Reduced Iron) hydrogen. It returns the category with the highest matching score 
    if it exceeds the threshold, otherwise returns None.
    
    Args:
        decarb_description (str): The description of the decarbonization technology to classify.
    
    Returns:
        str or None: The category name ('ccs', 'dri-hydrogen') or None if no match is found.
    """
    categories = {
        "ccs": ["ccs", "capture", "carbon capture", "sequestration", "co2 storage", "ccu", "CCU for BF-BOF"],
        "dri-hydrogen": ["hydrogen", "dri", "electrolysis", "h-dri", "direct reduced iron", "h2 dri", "green dri", "h2 production", "EAF"],
    }
    
    decarb_description = str(decarb_description).lower()
    
    best_match = None
    highest_score = 0
    
    for category, keywords in categories.items():
        match, score, __ = process.extractOne(decarb_description, keywords)
        if score > highest_score:
            highest_score = score
            best_match = category
    
    return best_match if highest_score > 80 else None  # Default to None if no strong match

def process_projects(df, cf_facility=0.8):
    """
    Processes a given DataFrame by applying classification, converting numeric columns, and filtering rows.
    
    This function:
        1. Applies the `classify_decarb_type` function to classify each project by its decarbonization technology.
        2. Converts relevant columns ('Steel production capacity' and 'Iron production capacity') to numeric values.
        3. Filters rows based on the presence of a valid decarbonization classification and positive production capacities.
    
    Args:
        df (pandas.DataFrame): The DataFrame containing project data to process.
    
    Returns:
        pandas.DataFrame: The filtered DataFrame after applying classification, numeric conversion, and row filtering.
    """
    # Apply classification and convert to numeric
    df["steel_decarb_classification"] = df["Technology to be used"].apply(classify_decarb_type)
    df["Steel production capacity (million tonnes per year)"] = pd.to_numeric(
        df["Steel production capacity (million tonnes per year)"], errors="coerce"
    )
    df["Iron production capacity (million tonnes per year)"] = pd.to_numeric(
        df["Iron production capacity (million tonnes per year)"], errors="coerce"
    )
    
    # Filter rows based on classification and production capacity
    df = df[(df["steel_decarb_classification"].notna()) & 
            ((df["Steel production capacity (million tonnes per year)"] > 0) | 
             (df["Iron production capacity (million tonnes per year)"] > 0))].copy()

    # Multiply capacity values by cf_facility first
    df['iron production'] = df['Iron production capacity (million tonnes per year)'].copy() * cf_facility
    df['steel production'] = df['Steel production capacity (million tonnes per year)'].copy() * cf_facility
    
    # Determine whether steel and/or iron are produced in a facility 
    df['steel_or_iron_production'] = df.apply(
        lambda row: 'both' if pd.notna(row['iron production']) and pd.notna(row['steel production'])
        else 'steel' if pd.notna(row['steel production'])
        else 'iron' if pd.notna(row['iron production'])
        else 'none',
        axis=1
    )
    
    df.rename(columns={'Country': 'Country/Area'}, inplace=True)
    return df

# Process both active and prospective projects
db_file_lc_projects = process_projects(db_file_lc_projects)

db_file_lc_projects = db_file_lc_projects.drop(
    columns=[col for col in db_file_lc_projects.columns if "Reference" in col])

db_file_lc_projects["power_classification"] = db_file_lc_projects["Technology details"].apply(classify_power_type)
db_file_lc_projects["steel_classification"] = 'unalloyed'

In [20]:
db_file_lc_projects['steel production'].sum()

46.48420480000001

In [21]:
db_file_lc_projects[['Internal ID', 'Country/Area', 'Project name', "steel_decarb_classification",'steel production']]

,Internal ID,Country/Area,Project name,steel_decarb_classification,steel production
6,GST-007,Germany,Hamburg H2,dri-hydrogen,NaN
9,GST-010,Australia,Whyalla Transformation Program,dri-hydrogen,1.440000
11,GST-012,Germany,tkH2Steel,dri-hydrogen,1.840000
12,GST-013,China,Paradigm project,dri-hydrogen,NaN
14,GST-015,Sweden,Oxelösund mini-mill,ccs,1.200000
15,GST-016,Austria,HYFOR,dri-hydrogen,NaN
17,GST-018,Australia,Green Metal Project,dri-hydrogen,NaN
18,GST-019,Sweden,Stegra Boden (formerly H2 Green Steel),dri-hydrogen,4.000000
19,GST-020,Germany,Bremen DRI (Steel4Future),dri-hydrogen,1.400000
20,GST-021,Germany,Eisenhüttenstadt DRI (Steel4Future),dri-hydrogen,1.400000


In [22]:
db_file_lc_projects['Start date'] = db_file_lc_projects['Year to be online']
db_file_lc_projects['Capacity operating status'] = db_file_lc_projects['Project status']

In [23]:
plants_mod_all = pd.concat([db_file_lc_projects, plants_mod], axis=0)
plants_mod_all['Start date'] = pd.to_numeric(plants_mod_all['Start date'], errors='coerce')
plants_mod_all.to_excel('data\merged_steel_dbs.xlsx')
plants_mod_all['power_classification'].value_counts()

power_classification
grid           878
pv             118
natural gas     20
nuclear         16
renewable        7
wind             5
coal             2
Name: count, dtype: int64